In [ ]:
import torch
from torch_geometric.utils import from_dgl, to_networkx, k_hop_subgraph, subgraph
import random
from torch_geometric.data import Data
from torch_geometric.utils import subgraph, to_undirected
from torch import Tensor
from typing import Optional, Tuple, Union
from collections import Counter
import numpy as np
from tqdm import tqdm
import torch_geometric.utils as utils

In [ ]:
reddit = torch.load('pyg_dataset/reddit.pt')
print(reddit)
anomaly_indices = torch.nonzero(reddit.y, as_tuple=False).squeeze().tolist()

In [ ]:
# set seed
seed = 0
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [ ]:
def random_walk_subgraph(pyg_graph, start_node, walk_length, max_nodes, onlyE=False):
    edge_index = to_undirected(pyg_graph.edge_index)

    # Extract a 2-hop subgraph around the start_node
    hop2_subset, hop2_edge_index, mapping, _ = k_hop_subgraph(start_node, num_hops=2, edge_index=edge_index, relabel_nodes=True)
    node_mapping = {i: hop2_subset[i].item() for i in range(len(hop2_subset))}
    if len(hop2_subset) > max_nodes:
        walks = []
        while len(set(walks)) < max_nodes:
            walk = random_walk(pyg_graph, start_node, walk_length)
            walks.extend(walk)
            
        subset = [item[0] for item in Counter(walks).most_common(max_nodes)]
        subg_edge_index, _ = utils.subgraph(subset, edge_index, relabel_nodes=True)
        node_mapping = {i: subset[i] for i in range(len(subset))}
    else:
        subset = hop2_subset
        subg_edge_index = hop2_edge_index

    x = pyg_graph.y[subset]
    x = torch.nn.functional.one_hot(x, num_classes=2).float()
    edge_attr = torch.tensor([[0, 1] for _ in range(subg_edge_index.shape[1])])
    extra_x = pyg_graph.x[subset]
    node_mapping = torch.tensor(list(node_mapping.values()))
    y = torch.empty(1, 0)
    # remove self-loops or not 
    if onlyE:
        x = torch.ones((len(subset), 1))
        
    # Create a new data object for the subgraph
    d = Data(x=x, edge_index=subg_edge_index, edge_attr = edge_attr, extra_x = extra_x,
             num_nodes=len(subset), node_mapping=node_mapping, y = y)
    return d

def random_walk(pyg_graph, start_node, walk_length=3):
    walk = [start_node]
    edge_index = pyg_graph.edge_index
    for _ in range(walk_length):
        neighbors = edge_index[1][edge_index[0] == walk[-1]]
        if len(neighbors) == 0:  # If no neighbors, stop the walk
            break
        next_node = np.random.choice(neighbors.cpu().numpy())
        walk.append(next_node)
    return walk

In [ ]:
i = 10
print(random_walk_subgraph(reddit, i, 3, 150, onlyE=True))
hop2_subset, hop2_edge_index, mapping, _ = k_hop_subgraph(i, num_hops=2, edge_index=reddit.edge_index, relabel_nodes=True)
print(len(hop2_subset))

In [ ]:
anomaly_subgraphs = []

for i in tqdm(range(1500)):
    node_idx = random.choice(anomaly_indices)
    subgraph = random_walk_subgraph(reddit, node_idx, 3, 150, onlyE=True)
    anomaly_subgraphs.append(subgraph)


torch.save(anomaly_subgraphs, f'./pyg_dataset/reddit_diffusion/reddit_anomaly.pt')


## Cluster-aware Sampling

In [1]:
from utils import GADDataset
data = GADDataset('reddit')

/home/allenwang/miniconda3/envs/gad_v2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data(edge_index=[2, 168016], train_masks=[10984, 20], val_masks=[10984, 20], test_masks=[10984, 20], num_nodes=10984, y=[10984], x=[10984, 64])


In [2]:
data.split(semi_supervised=False, trial_id=1)
data.cluster_anomalous_nodes(k=10)

tensor(4393) tensor(2196) tensor(4395)


100%|██████████| 146/146 [00:00<00:00, 748.81it/s]


getting subgraph embeddings


100%|██████████| 146/146 [03:23<00:00,  1.39s/it]

fitting kmeans


{1: [3073,
  9268,
  7223,
  8777,
  2647,
  9850,
  8843,
  9375,
  8876,
  8901,
  3783,
  2284,
  3314,
  9466,
  9989,
  7954,
  9491,
  5409,
  8486,
  8038,
  7028,
  6517,
  2954,
  6617,
  8667],
 7: [6149,
  5127,
  25,
  2591,
  3137,
  8261,
  6219,
  4689,
  7280,
  9845,
  3713,
  9887,
  9888,
  9391,
  5313,
  9926,
  3287,
  5357,
  7407,
  8952,
  9467,
  5381,
  9028,
  344,
  9569,
  5475,
  7044,
  8633,
  3530],
 4: [4614,
  2116,
  9322,
  644,
  8872,
  2228,
  216,
  6400,
  3855,
  6423,
  8983,
  1308,
  8478,
  6430,
  4902,
  7493,
  4956,
  2403,
  9616,
  8599,
  428,
  9141,
  7616,
  8651,
  9165],
 2: [4618,
  6167,
  7717,
  9777,
  2113,
  3667,
  1172,
  3228,
  6844,
  200,
  9934,
  8930,
  248,
  8448,
  9984,
  6919,
  1291,
  4884,
  7466,
  3883,
  829,
  6982,
  8559,
  6020,
  8589,
  8110,
  9699,
  501],
 3: [2587,
  1595,
  7760,
  2143,
  8320,
  4227,
  1673,
  6309,
  7375,
  724,
  9976,
  9479,
  6936,
  9078,
  2447,
  8602,
  7645,
